In [2]:
!pip install ahpy

In [3]:
from ahpy import ahpy
import pandas as pd
import numpy as np

In [68]:
df = pd.DataFrame({
    'Preco': np.random.randint(1, 101, size=10),
    'Produto': np.random.randint(1, 101, size=10),
    'Servico': np.random.randint(1, 101, size=10),
    'Documentacao': np.random.randint(1, 101, size=10),
    'Prazo': np.random.randint(1, 101, size=10),
    'Flexibilidade': np.random.randint(1, 101, size=10)
})

In [70]:
df

,Preco,Produto,Servico,Documentacao,Prazo,Flexibilidade
0,33,65,83,36,94,69
1,82,16,28,48,43,95
2,88,52,70,87,97,64
3,26,33,85,93,97,39
4,54,24,18,53,48,32
5,67,6,26,28,77,39
6,62,8,29,82,11,18
7,47,16,93,41,23,61
8,57,23,7,38,42,65
9,44,97,72,74,9,38


In [12]:
criteria_comparisons = {
                        ('Custo', 'Qualidade'): 1.2, ('Custo', 'Prazo'): 1.4,
			                  ('Qualidade', 'Prazo'): 1
                       }

In [14]:
criteria = ahpy.Compare('Criteria', criteria_comparisons, precision=4)
report = criteria.report(show=True)

{
    "name": "Criteria",
    "global_weight": 1.0,
    "local_weight": 1.0,
    "target_weights": {
        "Custo": 0.3932,
        "Qualidade": 0.3112,
        "Prazo": 0.2956
    },
    "elements": {
        "global_weights": {
            "Custo": 0.3932,
            "Qualidade": 0.3112,
            "Prazo": 0.2956
        },
        "local_weights": {
            "Custo": 0.3932,
            "Qualidade": 0.3112,
            "Prazo": 0.2956
        },
        "consistency_ratio": 0.0027
    }
}


In [23]:
criteria.local_weights

{'Custo': 0.3932, 'Qualidade': 0.3112, 'Prazo': 0.2956}

In [27]:
quality_comparisons = {
                    ('Produto', 'Servico'): 1, ('Produto', 'Documentacao'): 1.5,
			              ('Servico', 'Documentacao'): 1.4
                  }

In [28]:
quality_sub_criteria = ahpy.Compare('Criteria', quality_comparisons, precision=4)
quality_report = quality_sub_criteria.report(show=True)

{
    "name": "Criteria",
    "global_weight": 1.0,
    "local_weight": 1.0,
    "target_weights": {
        "Produto": 0.376,
        "Servico": 0.3675,
        "Documentacao": 0.2565
    },
    "elements": {
        "global_weights": {
            "Produto": 0.376,
            "Servico": 0.3675,
            "Documentacao": 0.2565
        },
        "local_weights": {
            "Produto": 0.376,
            "Servico": 0.3675,
            "Documentacao": 0.2565
        },
        "consistency_ratio": 0.0005
    }
}


In [35]:
quality_sub_criteria.local_weights

{'Produto': 0.376, 'Servico': 0.3675, 'Documentacao': 0.2565}

In [29]:
log_comparisons = {
                    ('Prazo', 'Flexibilidade'): 2
                  }

In [30]:
log_sub_criteria = ahpy.Compare('Criteria', log_comparisons, precision=4)
log_report = log_sub_criteria.report(show=True)

{
    "name": "Criteria",
    "global_weight": 1.0,
    "local_weight": 1.0,
    "target_weights": {
        "Prazo": 0.6667,
        "Flexibilidade": 0.3333
    },
    "elements": {
        "global_weights": {
            "Prazo": 0.6667,
            "Flexibilidade": 0.3333
        },
        "local_weights": {
            "Prazo": 0.6667,
            "Flexibilidade": 0.3333
        },
        "consistency_ratio": 0.0
    }
}


In [34]:
log_sub_criteria.local_weights

{'Prazo': 0.6667, 'Flexibilidade': 0.3333}

In [55]:
import pandas as pd
import numpy as np

criterios = pd.Series(criteria.local_weights)

custo = pd.Series({'Preco': 1})
qualidade = pd.Series(quality_sub_criteria.local_weights)
logistica = pd.Series(log_sub_criteria.local_weights)

In [61]:
criterios

Custo        0.3932
Qualidade    0.3112
Prazo        0.2956
dtype: float64

In [56]:
custo = custo * criterios.Custo
qualidade = qualidade * criterios.Qualidade
logistica = logistica * criterios.Prazo

In [75]:
pesos = pd.concat([custo, qualidade, logistica])

In [76]:
pesos

Preco            0.393200
Produto          0.117011
Servico          0.114366
Documentacao     0.079823
Prazo            0.197077
Flexibilidade    0.098523
dtype: float64

In [71]:
normalized_df = df.copy()
for column in df:
  normalized_df[column] = df[column] / df[column].sum()

normalized_df

,Preco,Produto,Servico,Documentacao,Prazo,Flexibilidade
0,0.058929,0.191176,0.162427,0.062069,0.173752,0.132692
1,0.146429,0.047059,0.054795,0.082759,0.079482,0.182692
2,0.157143,0.152941,0.136986,0.150000,0.179298,0.123077
3,0.046429,0.097059,0.166341,0.160345,0.179298,0.075000
4,0.096429,0.070588,0.035225,0.091379,0.088725,0.061538
5,0.119643,0.017647,0.050881,0.048276,0.142329,0.075000
6,0.110714,0.023529,0.056751,0.141379,0.020333,0.034615
7,0.083929,0.047059,0.181996,0.070690,0.042514,0.117308
8,0.101786,0.067647,0.013699,0.065517,0.077634,0.125000
9,0.078571,0.285294,0.140900,0.127586,0.016636,0.073077


In [84]:
weighted_df = normalized_df * pesos
weighted_df['peso_final'] = weighted_df.apply(lambda x: x.sum(), axis=1)
weighted_df.sort_values('peso_final', ascending=False)

,Preco,Produto,Servico,Documentacao,Prazo,Flexibilidade,peso_final
2,0.061789,0.017896,0.015667,0.011973,0.035335,0.012126,0.154786
0,0.023171,0.022370,0.018576,0.004955,0.034243,0.013073,0.116387
1,0.057576,0.005506,0.006267,0.006606,0.015664,0.017999,0.109618
3,0.018256,0.011357,0.019024,0.012799,0.035335,0.007389,0.104160
9,0.030894,0.033383,0.016114,0.010184,0.003279,0.007200,0.101054
5,0.047044,0.002065,0.005819,0.003854,0.028050,0.007389,0.094220
7,0.033001,0.005506,0.020814,0.005643,0.008378,0.011558,0.084900
8,0.040022,0.007915,0.001567,0.005230,0.015300,0.012315,0.082349
4,0.037916,0.008260,0.004029,0.007294,0.017486,0.006063,0.081047
6,0.043533,0.002753,0.006490,0.011285,0.004007,0.003410,0.071479


In [149]:
def rescale(x):
  return np.log(x) / np.log(x.max())
  return x / x.max()
weighted_df_rescaled = weighted_df.apply(rescale)
weighted_df_rescaled.sort_values('peso_final', ascending=False)

,Preco,Produto,Servico,Documentacao,Prazo,Flexibilidade,peso_final
6,1.125789,1.733963,1.300947,1.028882,1.651181,1.414074,1.414122
4,1.175412,1.410815,1.424116,1.129018,1.210451,1.270856,1.346794
8,1.155991,1.423334,1.668029,1.205355,1.250396,1.094461,1.338247
7,1.225280,1.530080,1.000000,1.187920,1.430533,1.110271,1.321897
5,1.097931,1.818582,1.329148,1.275423,1.069074,1.221614,1.266069
9,1.248972,1.000000,1.066096,1.052436,1.711211,1.228080,1.228540
3,1.437940,1.317145,1.023230,1.000000,1.000000,1.221614,1.212311
1,1.025365,1.530080,1.310010,1.151753,1.243357,1.000000,1.184935
0,1.352305,1.117752,1.029379,1.217760,1.009398,1.079596,1.152822
2,1.000000,1.183388,1.073372,1.015302,1.000000,1.098320,1.000000
